# Data Cleaning

### Importing Libraries:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt
%matplotlib inline

### Importing the Three CSV Files in this Dataset:
The three datasets are broken down into 'Crashes' (the main data file), and 'Vehicles'/'People' (secondary data files). The first task will be to inspect the two secondary files and identify any relevent columns to be imported into the main file.

In [ ]:
df_crashes = pd.read_csv('Data/Traffic_Crashes_-_Crashes.csv')
df_people = pd.read_csv('Data/Traffic_Crashes_-_People.csv',low_memory=False)
df_vehicles = df = pd.read_csv('Data/Traffic_Crashes_-_Vehicles.csv',low_memory=False) 

### Inspecting the 'Drivers' file:

In [ ]:
df_people.columns

There is a lot of overlap between this file and the main file. However the PERSON_TYPE, AGE, SEX, and PHYSICAL_CONDITION seem to contain new and relevant information. We will remove missing values, bin their rows into a binary categories and then encode them ordinally:

In [ ]:
#Removing Missing and Spurious Values:
df_people.SEX = df_people.SEX.replace(to_replace=['X','U'],value=np.nan)
df_people.PHYSICAL_CONDITION = df_people.PHYSICAL_CONDITION.replace(to_replace='REMOVED BY EMS',value=np.nan)
index_to_drop = df_people.loc[df_people.AGE == -49.0].index
index_to_drop1 = df_people.loc[df_people.AGE == -1.0].index
df_people = df_people.drop(index = index_to_drop)
df_people = df_people.drop(index = index_to_drop1)
df_people = df_people[['CRASH_RECORD_ID','PERSON_TYPE','AGE','SEX','PHYSICAL_CONDITION']]
df_people.SEX = df_people.SEX.fillna(method='pad')
df_people.PHYSICAL_CONDITION = df_people.PHYSICAL_CONDITION.fillna(method='pad')
df_people.AGE = df_people.AGE.fillna(method='pad')
df_people.AGE = df_people.AGE.fillna(method='bfill')

In [ ]:
#Binning the PHYSICAL_CONDIdtype columns into "Normal" and "Incapacitated" Bins:
def label_states (row):
    if row['PHYSICAL_CONDITION'] in ['NORMAL','OTHER']:
        return 'Normal'
    if row['PHYSICAL_CONDITION'] in ['IMPAIRED - ALCOHOL','HAD BEEN DRINKING','IMPAIRED - DRUGS','IMPAIRED - ALCOHOL AND DRUGS','FATIGUED/ASLEEP','EMOTIONAL',\
                                     'ILLNESS/FAINTED','MEDICATED']:
        return 'Incapacitated'
df_people['PHYSICAL_CONDTION'] = df_people.apply(lambda row: label_states(row), axis=1)

#Ordinally Encoding PHYSICAL_CONDITION and "SEX" columns
ord_enc = OrdinalEncoder()
condition_array = df_people.PHYSICAL_CONDITION.values.reshape(-1,1)
sex_array = df_people.SEX.values.reshape(-1,1)
df_people.SEX = ord_enc.fit_transform(sex_array)
df_people.PHYSICAL_CONDITION = ord_enc.fit_transform(condition_array)

#Grouping by "CRASH_RECORD_ID" and taking the mean so we can merge these columns with the main data file:
sex_df = df_people.SEX.groupby(df_people['CRASH_RECORD_ID']).mean()
age_df = df_people.AGE.groupby(df_people['CRASH_RECORD_ID']).mean()
cond_df = df_people.PHYSICAL_CONDITION.groupby(df_people['CRASH_RECORD_ID']).mean()

#Checking the shape to make sure they are all the same size:
sex_df.shape, age_df.shape, cond_df.shape

In [ ]:
#Concatenating sex_df, age_df, and cond_df into one dataframe that can merge with the main data file:
df_people_merge = pd.concat([sex_df,age_df,cond_df],axis=1)

### Inspecting the 'Vehicle' file:

Again, there seems to be a lot of overlap between this file and the main file. However the NUM_PASSENGERS and VEHICLE_DEFECT seem to contain new and relevant information. Just like the passenger data, we will remove missing values, bin their rows into a binary categories and then encode them ordinally:

In [ ]:
df_vehicles.FIRST_CONTACT_POINT.value_counts()

In [ ]:
#Filling missing values:
df_vehicles = df_vehicles[['CRASH_RECORD_ID','NUM_PASSENGERS','VEHICLE_DEFECT']]
df_vehicles['NUM_PASSENGERS'] = df_vehicles.NUM_PASSENGERS.fillna(method='pad')
df_vehicles['NUM_PASSENGERS'] = df_vehicles.NUM_PASSENGERS.fillna(method='bfill')
df_vehicles['VEHICLE_DEFECT'] = df_vehicles.VEHICLE_DEFECT.fillna(value='UNKOWN')

#Binning VEHICLE_DEFECT into "Normal" (1) and "Defective" (0) bins:
def label_states(row):
    if row['VEHICLE_DEFECT'] in ['NONE', 'UNKNOWN']:
        return 1
    if row['VEHICLE_DEFECT'] in ['OTHER','BRAKES','TIRES','STEERING','WHEELS','SUSPENSION','ENGINE/MOTOR','FUEL SYSTEM','LIGHTS','WINDOWS','CARGO','SIGNALS',\
                               'RESTRAINT SYSTEM','TRAILER COUPLING']:
        return 0
df_vehicles['VEHICLE_DEFECT'] = df_vehicles.apply(lambda row: label_states(row), axis=1)

#Grouping by "CRASH_RECORD_ID" and taking the mean so we can merge these columns with the main data file:
df_num_passengers = df_vehicles.NUM_PASSENGERS.groupby(df_vehicles.CRASH_RECORD_ID).mean()
df_defect = df_vehicles.VEHICLE_DEFECT.groupby(df_vehicles.CRASH_RECORD_ID).mean()

#Checking the shape to make sure they are all the same size:
df_num_passengers.shape, df_defect.shape

In [ ]:
#Concatenating sex_df, age_df, and cond_df into one dataframe that can merge with the main data file:
df_vehicles_merge = pd.concat([df_num_passengers,df_defect],axis=1)

### Merging the Three Data Files:

Now that we have cleaned the secondary data files and selected the rows and columns we want, we need to merge the three dataframes on  "CRASH_RECORD_ID":

In [ ]:
#Merging the two secondary files first:
vehicles_people = pd.merge(df_vehicles_merge, df_people_merge, how='outer', on='CRASH_RECORD_ID')
#Merging the secondary files to the primary file:
df = pd.merge(df_crashes, vehicles_people, how='outer', on='CRASH_RECORD_ID')

In [ ]:
df.shape

In [ ]:
nulls = pd.DataFrame(df.isnull().sum())
percent_null = []

In [ ]:
for i,j in nulls.items():
    percent_null.append(j/(df.shape[0]))

In [ ]:
percent_null = pd.DataFrame(percent_null)
percent_null = percent_null.T
percent_null.loc[percent_null[0] > 0.5]

In [ ]:
df.info()

### Dropping Columns
This dataset contains columns that have large numbers of missing values, or do not bear any relevance to the classification target. Below is a table outlining the columns dropped and why:

|Column|Reason Dropped|Column|Reason Dropped
|------|--------------|------|--------------
|PHOTOS_TAKEN_I|Too many missing values|CRASH_DATE_EST_I|Too many missing values/Not relevant
|STATEMENTS_TAKEN_I|Too many missing values/Not relevant|INJURIES_UNKOWN| Essentially a null column|
|DOORING_I|Too many missing values|REPORT_TYPE|Not relevant|                        
|WORK_ZONE_I|Too many missing values|RD_NO|Only unique values|                    
|WORK_ZONE_TYPE|Too many missing values|DATE_POLICE_NOTIFIED|Not relevant|                   
|WORKERS_PRESENT_I|Too many missing values|NOT_RIGHT_OF_WAY_I|Too many missing values|
|LANE_CNT|Too many missing values|STREET_DIRECTION|Not relevant|
|INTERSECTION_RELATED|Too many missing values|HIT_AND_RUN|Too many missing values
|SEC_CONTRIBUTORY_CAUSE|Too many missing values|num_units|Not relevant|

In [ ]:
#Dropping Columns
df = df.drop(['PHOTOS_TAKEN_I','STATEMENTS_TAKEN_I','DOORING_I','WORK_ZONE_I','WORK_ZONE_TYPE','WORKERS_PRESENT_I','LANE_CNT','CRASH_DATE_EST_I',\
              'REPORT_TYPE','RD_NO','DATE_POLICE_NOTIFIED','NOT_RIGHT_OF_WAY_I','STREET_DIRECTION','INJURIES_UNKNOWN','INTERSECTION_RELATED_I',\
              'HIT_AND_RUN_I','NUM_UNITS'],axis=1)

### Inspecting the type and count of values within all columns:

In [ ]:
# # Checking the value_counts of all remaining columns:
# for i,j in df.items():
#     print('------')
#     print(df[i].value_counts())

### Dropping Null values: 
There are multiple columns with string values such as 'UNKNOWN' ,'UNABLE TO DETERMINE' etc. These are essentially null values for those columns as they contain no information.

In [ ]:
#Visualizing location of null values:
import missingno as msno
msno.matrix(df)
plt.show()

In [ ]:
##Dropping Rows with null values:
df = df.dropna()

### Converting to Lowercase:
So that we're not constantly pressings caps lock or shift, it is useful to convert all column names and strings to lowercase:

In [ ]:
#Making things lowercase
df.columns = df.columns.map(lambda x: x.lower())
df_str = df.select_dtypes(include=object)
df_num = df.select_dtypes(exclude=object)
df_str = df_str.astype(str)
df_str = df_str.applymap(lambda x: x.lower())
df = pd.concat([df_str, df_num], axis=1)

### Converting 'crash_date' Column to a DateTime object:
As a consequnce of the previous cell of code, the 'crash_date' column was converted into a string, it is probably more useful to have this column stored as a DateTime object:

In [ ]:
#Setting crash_date as DateTime Object
df.crash_date = pd.to_datetime(df.crash_date)

### Checking for Duplicates:

In [ ]:
dup = df.duplicated()
dup.value_counts()
#There appear to be no duplicates

### Saving Cleaned Dataframe:

In [ ]:
df.info()

In [ ]:
df.to_csv('Traffic_Crashes_Cleaned.csv')